### Getting Started
 This tutorial provides a starting point for getting acquainted with NeuroDOT_Py.<br><br>     
                                                 
 You can download the notebook from the [NeuroDOT_Py GitHub](https://github.com/WUSTL-ORL/NeuroDOT_py)                                 <br><br>

In [ ]:
%%capture
# Install all required libraries for running NeuroDOT (only needs to be done on first use)
!pip3 install -r ../../requirements.txt

In [ ]:
# General imports
import sys
import math
import os
import shutil
import json

import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.colors as colors
import scipy.interpolate
import numpy.matlib as mlb
import numpy.matlib as nm

from math import trunc
from pickle import NONE
from numpy import float64, matrix
from numpy.lib.shape_base import expand_dims
from matplotlib.pyplot import colorbar, colormaps
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.mplot3d import Axes3D

# Add NeuroDOT library to the path
import neuro_dot as ndot

In [ ]:
# To get started, edit this cell for folder paths; EDIT AT THE BEGINNING ONLY
saveImages = 'yes' # Options are yes/no to save output figures (must be lowercase 'yes' or 'no')
saveImagePath ="./Output" # Path to folder to save output, figures will be saved to subdirectory called 'Output_Images'
saveNoteBookPath = "./Output" # Path to folder to save output, Jupyter Notebook will be saved to subdirectory called 'Output_Notebook'

In [ ]:
# To get started, edit this cell for folder paths; EDIT AT THE BEGINNING ONLY
saveImages = 'yes' # Options are yes/no to save output figures (must be lowercase 'yes' or 'no')
saveImagePath ="./outputfiles" # Path to folder to save output, figures will be saved to subdirectory called 'Output_Images'
saveNoteBookPath = "./outputfiles" # Path to folder to save output, Jupyter Notebook will be saved to subdirectory called 'Output_Notebook'

# RUN THIS CELL ONLY ONCE IF YOU CHOOSE TO RE RUN THE CELL ABOVE FIRST
if saveImages == 'yes':
    saveImagePath = saveImagePath + '/Output_Images/'
    if not os.path.exists(saveImagePath): 
        os.makedirs(saveImagePath)
    # The code below is only relevant for use with XNAT container
    saveNoteBookPath = saveNoteBookPath + '/Output_Notebook/'
    if not os.path.exists(saveNoteBookPath):
        os.makedirs(saveNoteBookPath)
else:
    print('SAVED IMAGES IS SET TO NO. NO IMAGES WILL BE SAVED.')

In [ ]:
# TO MAKE CHANGES TO FIGURES, EDIT ONLY THIS CELL
# patient_data = "/input/subjectdata.mat"  # Path to data; an example data set has been provided here
participant_data = 'NeuroDOT_Data_Sample_CCW1'
# params_file = "./Data/params.txt"
a = ""
# with open(params_file) as f:
#     for line in f: 
#         a = a + line.strip()
# params = json.loads(a)
# # If you want to change an individual or a few params, just follow syntax below (NOTE: make changes before 'print' statements to confirm dictionary updates): 
# # params['bthresh'] = 0.075
# print("Params being used: \n")
# print(params)
params = {'bthresh':0.075,'det':1, 'highpass':1, 'lowpass1':1, 'ssr':1, 'lowpass2':1, 'DoGVTD':1, 'resample': 1, 'omega_hp': 0.02, 'omega_lp1': 1, 'omega_lp2': 0.5,'freqout': 1, 'rstol': 1e-5 ,'DQC_ONLY': 0, 'omega_resample': 1} 

In [ ]:
# Load your data file, or one of the provided data samples using the absolute path to the file (.mat format)
data_path = os.path.abspath(os.path.join(sys.path[0],"../..", 'Data', participant_data))
# data_path = "/Users/peellelab/NeuroDOT_py_ABmac/NeuroDOT_py/Data/NeuroDOT_Data_Sample_CCW1.mat"
# data_path = os.path.join('../../',os.path.dirname(sys.path[0]),'Data',participant_data)
data = ndot.loadmat(data_path)['data'] 
__info = ndot.loadmat(data_path)['info'] 
tag = participant_data.split("/")[-1].split("_")[-1][:-4]
flags = ndot.loadmat(data_path)['flags']

In [ ]:
# Set parameters for A and block length for quick processing examples
if 'CCW1' in participant_data or 'CCW2' in participant_data or 'CW1' in participant_data or 'OUT' in participant_data:
    params['dt']=36                     # Block length
    params['tp']=16                     # Example (block averaged) time point
if 'IN1' in participant_data:
    params['dt']=36                     # Block length
    params['tp']=32                     # Example (block averaged) time point

if  'HW1'in participant_data or'HW2' in participant_data or'RW1' in participant_data or'GV1'in participant_data or'HW3_Noisy' in participant_data:
    params['dt']=30                     # Block length
    params['tp']=16                     # Example (block averaged) time point

# Make sure params correspond to the data type
print('dt:', params['dt'])
print('tp:', params['tp'])

In [ ]:
params_time_traces = params.copy()
params_metricsII = params.copy()
params_cap = params.copy()

ndot.Plot_RawData_Time_Traces_Overview(data,__info, params_time_traces)   # Time traces
filename = saveImagePath +'Time_Traces_Overview' +'_' + tag + '.png'
if saveImages == 'yes':
    plt.savefig(filename,format = 'png')

ndot.Plot_RawData_Metrics_II_DQC(data,__info, params_metricsII)         # Spectrum, falloff, and good signal metric
filename = saveImagePath +'RawData_Metrics_II' +'_' + tag + '.png'
if saveImages == 'yes':
    plt.savefig(filename,format = 'png')
    
__info_out = ndot.Plot_RawData_Cap_DQC(data,__info, params_cap)   # Cap-relevant views
filename = saveImagePath +'RawData_Cap_DQC' +'_' + tag + '.png'
if saveImages == 'yes':
    plt.savefig(filename,format = 'png')

In [ ]:
## Logmean Light Levels
lmdata = ndot.logmean(data)[0]

In [ ]:
## Detect Noisy Channels
info = ndot.FindGoodMeas(lmdata, __info, 0.075)

# Example visualization
keep = np.logical_and(np.logical_and(np.where(info['pairs']['WL'] == 2,1,0), np.where(info['pairs']['r2d'] < 40,1,0)), info['MEAS']['GI']) # measurements to include

ndot.DynamicFilter(data,info, params,'fft_lml', saveImages,saveImagePath)

In [ ]:
## Show nn1, nn2, nn3 (plots)
ndot.nlrGrayPlots_220324(lmdata,info)
filename = saveImagePath +'nlrGrayPlots_lml' +'_' + tag + '.png'

if saveImages == 'yes':
    plt.savefig(filename,format = 'png')

In [ ]:
ndot.DynamicFilter(data,info, params,'lml', saveImages,saveImagePath)

In [ ]:
## Detrend and High-pass Filter the Data
ndot.DynamicFilter(data,info, params,'fft_dt_hp', saveImages,saveImagePath)

In [ ]:
ndot.DynamicFilter(data,info, params,'high_pass', saveImages,saveImagePath)

In [ ]:
## Low Pass Filter 1
ndot.DynamicFilter(data,info, params,'low_pass_fft', saveImages,saveImagePath)

In [ ]:
ndot.DynamicFilter(data,info, params,'low_pass', saveImages,saveImagePath)

In [ ]:
## Superficial Signal Regression
ndot.DynamicFilter(data,info, params,'fft_superficial_signal', saveImages,saveImagePath)

In [ ]:
# Generate 2 subplots
fig = plt.figure(dpi = 100, tight_layout = True)
ddata = ndot.detrend_tts(lmdata)
hpdata = ndot.highpass(ddata, params['omega_hp'], info['system']['framerate'])
lp1data = ndot.lowpass(hpdata, params['omega_lp1'], info['system']['framerate']) 
hem = ndot.gethem(lp1data, info)
gs = gridspec.GridSpec(2,1)
ax1 =  plt.subplot(gs[0,0])
ax2 =  plt.subplot(gs[1,0])

ax1.plot(hem[1,:], linewidth = 0.5)
ax1.set_title('Estimated common superficial signal')
ax1.set_xlabel('Time (samples)')
arr1 = hem[1,:]
arr = np.reshape(arr1, (1,np.size(arr1)))
ftdomain,ftmag,_,_ = ndot.fft_tts(arr,info['system']['framerate']) 
ftmag = np.reshape(ftmag, (len(ftdomain)))
ax2.semilogx(ftdomain,ftmag, linewidth = 0.5)
ax2.set_xlabel('Frequency (Hz)')
ax2.set_ylabel('|X(f)|') # plot vs. log frequency
ax2.set_xlim([1e-3, 10])

filename = saveImagePath +'fft_SSR'+'_' + tag + '.png'
if saveImages == 'yes':
    plt.savefig(filename, facecolor = 'white',format = 'png')

In [ ]:
ndot.DynamicFilter(data,info, params,'superficial_signal', saveImages,saveImagePath)

In [ ]:
## Low Pass Filter 2
ndot.DynamicFilter(data,info,params,'fft_low_pass2', saveImages,saveImagePath)

In [ ]:
info_new =ndot.DynamicFilter(data,info,params, 'fft_resample', saveImages,saveImagePath)

In [ ]:
info_new = ndot.DynamicFilter(data,info, params,'resample', saveImages,saveImagePath)

In [ ]:
info_new = ndot.DynamicFilter(data,info,params, 'fft_ba', saveImages,saveImagePath)

In [ ]:
info_new = ndot.DynamicFilter(data,info,params, 'ba',saveImages,saveImagePath)